In [1]:
from sklearn.metrics import mean_absolute_percentage_error as mape
from all_functions import *
from darts import TimeSeries
import os
from darts.models import StatsForecastAutoETS
from darts.models import StatsForecastAutoARIMA
import statsmodels.api as sm
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
def percorrer_csv(diretorios, save_dir='./', horizon=5):
    cont = 0
    
    resultado_file = save_dir+'/result_ets.csv'
    train = pd.Series()
    for diretorio in diretorios.split():
        if os.path.isdir(diretorio):
            for root, _, arquivos in os.walk(diretorio):
                for arquivo in arquivos:
                    if arquivo.endswith('.csv'):
                        caminho_arquivo = os.path.join(root, arquivo)
                        try:
                            paths = caminho_arquivo.split('/')
                            file_csv = paths[-1]
                            estado = file_csv.split('_')[2]
                            derivado = file_csv.split('_')[3].replace('.csv', '')
                            cidade = file_csv.split('_')[1]
                            df = pd.read_csv(caminho_arquivo, sep=";")
                            df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y')

                            df.set_index('timestamp', inplace=True)
                            start_timestamp = '1998-01-01'  # ou qualquer data inicial que você queira
                            index_series = pd.date_range(start=start_timestamp, periods=len(df), freq='Y')
                            # series = pd.Series(series_value, index=index_series)
                            serie_m3 = pd.Series(df['m3'].values, index=index_series)
                            train, test = train_test_stats(serie_m3, horizon)
                            w = len(train)
                            # train_tf, _, _ = rolling_window_series(train, horizon)
                            train_tf = znorm(train)
                            train_darts = TimeSeries.from_series(train_tf)
                            
                            if len(serie_m3) > 12:
                                test_index = test
                                #roda 5 vezes para 5 anos
                                preds_final = []
                                preds_series = pd.Series()
                                train_concat = train
                                # model = StatsForecastAutoETS(model='AAN')
                                # model = StatsForecastAutoARIMA(season_length=1)
                                model = ETSModel(train_tf, trend='add', damped_trend=True, seasonal_periods=1, freq='A').fit(full_output=False, disp=False)


                                predictions = model.forecast(steps=horizon)
                                preds_norm = pd.Series(predictions, index=test.index)
                                # model.fit(train_darts)
                                # result = model.predict(n=horizon)
                                # preds_norm = pd.Series(result.values().flatten().tolist(), index=test.index)
                                preds_real = reverse_transform_norm_preds(preds_norm, train_concat, "normal", w=horizon)
                                preds_final = preds_real.values
                                # for i in range(5):
                                #     model = StatsForecastAutoETS()
                                #     model.fit(train_darts)
                                #     result = model.predict(n=1)
                                #     new_index = test_index.index[0] + pd.DateOffset(years=i)
                                #     new_index = pd.DatetimeIndex([new_index])
                                #     preds_norm = pd.Series(result.values().flatten().tolist(), index=new_index)
                                #     w = len(train_concat)
                                #     preds_real = reverse_transform_norm_preds(preds_norm, train_concat, "normal", w=w)
                                #     train_concat = pd.concat([train_concat, preds_real])
                                #     preds_final.extend(preds_real.values)
                                #     if preds_series.empty:
                                #         preds_series = preds_real
                                #     else:
                                #         preds_series = pd.concat([preds_series, preds_real])
                                    
                                #     #concat train com preds_real
                                #     # train_concat_tf, _, _ = rolling_window_series(train_concat, w)
                                #     train_concat_tf = znorm(train_concat)
                                #     train_darts = TimeSeries.from_series(train_concat_tf)
                                    
                                cont+=1
                                mape_result = mape(test.values, preds_final)
                                pocid_result = pocid(test.values, preds_final)
                                dados = {
                                    'PRODUCT': derivado,
                                    'UF': estado.upper(),
                                    'MODEL': 'ETS',
                                    'PARAMETERS': str({'trend':'add', 'damped_trend':True, 'seasonal_periods':1, 'freq':'A'}),
                                    # 'mape': mape_result,
                                    # 'pocid': pocid_result,
                                    'city': cidade,
                                    'test': [list(test.values)],
                                    'preds': [list(preds_final)],
                                    # 'index_test': [test.index],
                                    # 'index_preds': [preds_series.index]
                                    
                                }
                                df_result = pd.DataFrame(dados)
                                df_result = df_result.reset_index(drop=True)
                                print(cont)

                                if not os.path.exists(resultado_file):
                                    df_result.to_csv(resultado_file, sep=';', index=False, mode='w', header=True)
                                else:
                                    df_result.to_csv(resultado_file, sep=';', index=False, mode='a', header=False)
                        except Exception as e:
                            print(f"Erro ao ler {arquivo}: {e}")
        else:
            print(train)
            print(f"O diretório {diretorio} não existe ou não é uma pasta válida.")




/home/user/miniconda3/envs/lucas/lib/python3.11/site-packages/optuna/study/_optimize.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from optuna import progress_bar as pbar_module


In [49]:
import numpy as np
import pandas as pd

pd.options.display.float_format = "{:.4f}".format

model = "FT_catboost"
part = '/normal'
csv_files = [
    f"./timeseries/mestrado/resultados/{model}{part}/ANP_MONTHLY.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/AUSTRALIAN_ELECTRICITY_DEMAND_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/M4_HOURLY_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/M4_WEEKLY_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/NN5_DAILY_DATASET_WITHOUT_MISSING_VALUES.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/NN5_WEEKLY_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/PEDESTRIAN_COUNTS_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/US_BIRTHS_DATASET.csv",
]

for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file, sep=";")
        print(f"{csv_file.split('/')[-1]}: {len(df)} entries")
    except Exception as e:
        print(f"Erro ao ler {csv_file}: {e}")

ANP_MONTHLY.csv: 5642 entries
AUSTRALIAN_ELECTRICITY_DEMAND_DATASET.csv: 20 entries
M4_HOURLY_DATASET.csv: 1656 entries
M4_WEEKLY_DATASET.csv: 1436 entries
NN5_DAILY_DATASET_WITHOUT_MISSING_VALUES.csv: 444 entries
NN5_WEEKLY_DATASET.csv: 444 entries
PEDESTRIAN_COUNTS_DATASET.csv: 264 entries
US_BIRTHS_DATASET.csv: 4 entries


In [2]:
import numpy as np
import pandas as pd

pd.options.display.float_format = "{:.4f}".format
df_agent = pd.read_csv(
    "./timeseries/mestrado/resultados/ADE/ANP_MONTHLY.csv",
    sep=";",
    )
# pegar cada valor das colunas mape, pocid, smape, rmse, msmape, mae e gerar uma media de todos
df_agent = df_agent[df_agent['final_test'] == '2024-11-30']

print(len(df_agent))
print(
    df_agent[["mape", "pocid", "smape", "rmse", "msmape", "mae"]]
    .apply(pd.to_numeric, errors="coerce")
    .mean()
)

182
mape        0.1688
pocid      67.6324
smape       0.1071
rmse     4455.8191
msmape      0.1071
mae      3761.5651
dtype: float64


In [3]:
df_agent = pd.read_csv(
    "./timeseries/mestrado/resultados/BEST_MEAN_EACH/ANP_MONTHLY.csv",
    sep=";",
)
# pegar cada valor das colunas mape, pocid, smape, rmse, msmape, mae e gerar uma media de todos
df_agent = df_agent[df_agent['final_test'] == '2024-11-30']

print(len(df_agent))
print(
    df_agent[["mape", "pocid", "smape", "rmse", "msmape", "mae"]]
    .apply(pd.to_numeric, errors="coerce")
    .mean()
)

182
mape     4922596738537183.0000
pocid                  70.3297
smape                   0.1725
rmse                 5086.3435
msmape                  0.1724
mae                  4209.5350
dtype: float64


In [6]:
df_agent = pd.read_csv(
    "./timeseries/mestrado/resultados/simple_selective_agent_qwen3=14b/ANP_MONTHLY.csv",
    sep=";",
)
# pegar cada valor das colunas mape, pocid, smape, rmse, msmape, mae e gerar uma media de todos
df_agent = df_agent[df_agent['final_test'] == '2024-11-30']

print(len(df_agent))
print(
    df_agent[["mape", "pocid", "smape", "rmse", "msmape", "mae"]]
    .apply(pd.to_numeric, errors="coerce")
    .mean()
)

180
mape     2010481933651979.0000
pocid                  62.7273
smape                   0.2310
rmse                 6471.4706
msmape                  0.2308
mae                  5652.2558
dtype: float64


In [ ]:
[3983.0277899919993, 3973.07890045, 3866.6295095660003, 3828.631270392, 3729.2696203879996, 3452.1812429240003, 3390.6756850620004, 3423.8267452640002, 3505.0979301520006, 3611.051559026, 3758.3487575619997, 3674.512047092]

[4032.51, 3992.17, 4059.04, 4247.48, 4254.99, 4323.32, 4323.39, 4411.27, 4414.45, 4362.44, 4354.62, 4320.34]


In [7]:
st_model = [4228.67952219, 4184.02515999, 4348.57974052, 4646.88140888, 4788.81122345,
 4824.75224054, 4743.52244105, 4671.57563018, 4319.82683479, 4243.40635961,
 4252.19789882, 4321.99214161]
nv_model = [3338.97316667, 3362.96009722, 3412.54843866, 3483.21914188, 3482.90423704,
 3529.10209012, 3594.69143097, 3676.08238355, 3747.21424884, 3722.81543624,
 3669.75588926, 3606.89971337]
rf_model = [4498.30869147, 4186.80134844, 4343.8227019,  4804.43880326, 4636.96343345,
 4630.66272281, 4339.02323765, 4703.34133693, 4967.68246514, 4829.55202694,
 4814.46705323, 4569.91170854]
catboost_model = [4326.13155613, 4382.55534905, 4509.89277512, 4686.47021107, 4816.48914307,
 5084.26302361, 5364.61752476, 5409.61015885, 5369.5963819,  5375.62555846,
 5415.51436694, 5465.75757413]
svr_model = [3770.45019031, 3844.50019776, 3680.37313315, 3616.40043878, 3549.76118581,
 3547.83615925, 3575.08615114, 3595.71791131, 3667.90595047, 3640.80891525,
 3621.14187752, 3637.16139457]

all_preds = [st_model, nv_model, rf_model, catboost_model, svr_model]
np.mean(all_preds, axis=0)

array([4032.50862535, 3992.16843049, 4059.04335787, 4247.48200077,
       4254.98584456, 4323.32324727, 4323.38815711, 4411.26548416,
       4414.44517623, 4362.4416593 , 4354.61541715, 4320.34450644])